# Exploration Data des Villes pour Open Data des Communes 

In [6]:
import requests
import pandas as pd
import geopandas as gpd
import os

In [7]:
def download_file(url, filename):
    if filename in os.listdir():
        print("Fichier déjà présent")
        return None
    response = requests.get(url)
    with open(filename, 'wb') as f:
        f.write(response.content)
    print('Fichier téléchargé')

In [8]:
def get_df(url, filename, encoding='utf-8'):
    download_file(url, filename)
    print("DataFrame prête")
    return pd.read_csv(filename, encoding=encoding, sep=';')

In [9]:
def nb_par_commune(df, colname="commune"):
    """Groups a DataFrame per column colname (the commune) and counts rows, renames the one column, and sorts the df before returning it"""
    return df.groupby(colname)[["nom"]].count().rename(columns={"nom": "nombre"}).sort_values(by="nombre", ascending=False)
    

## Bron

## Les Questions : 
1. La ville de Bron dispose de 4 stations de Vélov' (source : [data.grandlyon.com](https://download.data.grandlyon.com/ws/rdata/jcd_jcdecaux.jcdvelov/all.csv?maxfeatures=-1) via JCDecaux)
1. 3.6% de la superficie de Bron est occupée par ses Parcs et Jardins (source : [data.grandlyon.com](https://data.grandlyon.com))
1. La température la plus basse jamais enregistrée à Bron est de -24,6°C, en 1938 (source : [donneespubliques.meteofrance.fr](https://donneespubliques.meteofrance.fr/))

In [10]:
velov_url = 'https://download.data.grandlyon.com/ws/rdata/jcd_jcdecaux.jcdvelov/all.csv?maxfeatures=-1'
velov_filename = "velov.csv"

In [11]:
velov = get_df(velov_url, velov_filename)

Fichier déjà présent
DataFrame prête


In [12]:
velov.groupby('commune').count()

,number,name,address,address2,address_jcd,nmarrond,bonus,pole,bike_stands,available_bike_stands,...,etat,nature,titre,description,startdate,enddate,overflow,total_stands,main_stands,overflow_stands
commune,,,,,,,,,,,,,,,,,,,,,
ALBIGNY-SUR-SAONE,1,1,1,0,0,0,1,1,1,1,...,0,0,0,0,0,0,1,1,1,0
BRON,4,4,4,1,0,0,4,4,4,4,...,0,0,0,0,0,0,4,4,4,0
CALUIRE-ET-CUIRE,5,5,5,1,3,3,5,3,5,5,...,0,0,0,0,0,0,5,5,5,0
COLLONGES-AU-MONT-D'OR,1,1,1,0,0,0,1,1,1,1,...,0,0,0,0,0,0,1,1,1,0
COUZON-AU-MONT-D'OR,1,1,1,0,0,0,1,1,1,1,...,0,0,0,0,0,0,1,1,1,0
DECINES-CHARPIEU,1,1,1,0,0,0,1,1,1,1,...,0,0,0,0,0,0,1,1,1,0
ECULLY,1,1,1,1,0,0,1,1,1,1,...,0,0,0,0,0,0,1,1,1,0
FONTAINES-SUR-SAONE,2,2,2,0,0,0,2,1,2,2,...,0,0,0,0,0,0,2,2,2,0
LA MULATIERE,1,1,1,0,0,0,1,1,1,1,...,0,0,0,0,0,0,1,1,1,0


In [13]:

bron_parcs_et_jardins_url = "https://download.data.grandlyon.com/ws/grandlyon/bron.parcjardin_latest/all.csv?maxfeatures=-1"
bron_parcs_et_jardins_file = "bron_parcs_et_jardins.csv"


In [14]:
bron_superficie_m2 = 1010 * 10_000

In [15]:
bron_parcs_df = get_df(bron_parcs_et_jardins_url, bron_parcs_et_jardins_file)

Fichier déjà présent
DataFrame prête


In [16]:
bron_parcs_df.surf_tot_m2 = bron_parcs_df.surf_tot_m2.apply(lambda s: int(s[:-2]))

In [17]:
bron_parcs_superficie_m2 = bron_parcs_df.surf_tot_m2.sum()

In [18]:
bron_part_parcs = bron_parcs_superficie_m2 / bron_superficie_m2

In [19]:
bron_part_parcs

0.0367029702970297

## Corbas
### Les Questions
1. Corbas a 3094 adresses répertoriés dans la Base d'Adresses Nationale (source : [Base d'Adresse Nationale](https://adresse.data.gouv.fr/base-adresse-nationale/69273#12.93/45.6655/4.91))
1. Corbas a 43 équipements sportifs, ce qui la place devant Lyon 1er (source : [data.grandlyon.com](https://download.data.grandlyon.com/ws/rdata/urbalyon.recenseqptsport/all.csv?maxfeatures=-1))
1. Le nom de la commune viendrait de corbier, dérivé du mot cormier et non du corbeau. (source: Jean Coste, Dictionnaire des noms propres : toponymes et patronymes de France : quelles origines, quelles significations ?, Paris, Armand Colin, novembre 2006, 701 p. ([ISBN 2-200-26969-2](https://catalogue.bnf.fr/ark:/12148/cb40945028f)), p. 132.)

In [24]:
equipements_sportifs_url = "https://download.data.grandlyon.com/ws/rdata/urbalyon.recenseqptsport/all.csv?maxfeatures=-1"
equipements_sportifs_filename = "equipements_sportifs.csv"
equipements = get_df(equipements_sportifs_url, equipements_sportifs_filename)

Fichier déjà présent
DataFrame prête


In [25]:
equipements[equipements.commune.str.contains('Corbas')].shape

(43, 15)

In [26]:
equipements_classement = equipements.groupby('commune').count()[['nom']].sort_values('nom', ascending=False)

In [27]:
equipements_classement['ordre'] = range(1, equipements_classement.shape[0] +1)

In [28]:
equipements_classement.iloc[:30]

,nom,ordre
commune,,
Villeurbanne,288,1
Lyon 8e Arrondissement,151,2
Lyon 9e Arrondissement,146,3
Vénissieux,141,4
Lyon 7e Arrondissement,139,5
Vaulx-en-Velin,127,6
Bron,125,7
Lyon 5e Arrondissement,115,8
Saint-Priest,102,9


## La Tour de Salvagny
### Les Questions
1. Il y 16 arrêts de car Sytral à la Tour-de-Salvagny (Source : [data.grandlyon.com](https://download.data.grandlyon.com/ws/rdata/cdr_cardurhone.cdrarret/all.csv?maxfeatures=-1))
1. La Tour-de-Salvagny a 54 installations Photo_Voltaïques (Source : [data.grandlyon.com](https://download.data.grandlyon.com/ws/rdata/nrj_energie.nrjinstallphotovolt_1/all.csv?maxfeatures=-1))
1. La population de la commune a décuplé depuis la Révolution Française (Sources : [École des Hautes Études en Sciences Sociales](http://cassini.ehess.fr/fr/html/fiche.php?select_resultat=37913) et [INSEE](https://www.insee.fr/fr/statistiques/6005800?geo=COM-69250))

In [20]:
# data cars du rhone
cars_csv_url = "https://download.data.grandlyon.com/ws/rdata/cdr_cardurhone.cdrarret/all.csv?maxfeatures=-1"
cars_csv_file = 'cars.csv'

In [21]:
cars = get_df(cars_csv_url, cars_csv_file)

Fichier déjà présent
DataFrame prête


In [22]:
cars_la_tour_de_salvagny = cars[cars.nom.str.startswith('LA TOUR')]

In [23]:
cars_la_tour_de_salvagny.shape

(16, 5)

In [24]:
installations_photo_voltaiques_url = "https://download.data.grandlyon.com/ws/rdata/nrj_energie.nrjinstallphotovolt_1/all.csv?maxfeatures=-1"
installations_photo_voltaiques_file = 'installations_voltaiques.csv'

In [25]:
installations_photo_voltaiques = get_df(installations_photo_voltaiques_url, installations_photo_voltaiques_file)

Fichier déjà présent
DataFrame prête


In [26]:
installs_la_tour = eval(installations_photo_voltaiques[installations_photo_voltaiques.commune.str.contains('Tour')].data[39])

In [27]:
installs_la_tour[-1].get('nbinstall')

54

In [28]:
demographie_url = "https://download.data.grandlyon.com/ws/grandlyon/ter_territoire.terpopulation_latest/all.csv?maxfeatures=-1"
demographie_file = "demographie.csv"
download_file(demographie_url, demographie_file)


Fichier déjà présent


In [29]:
demographie = pd.read_csv(demographie_file, sep=';')

In [30]:
demographie.loc[demographie.commune.str.contains('Tour'), 'pop2018']

27    4137
Name: pop2018, dtype: int64

## Pierre-Bénite
### Les Questions
1. Le nom de Pierre-Bénite fait référence à une roche qui affleure à cet endroit du Rhône, en bas de la butte Haute-Roche. La première référence à cette roche date de 1339, où elle est qualifiée de Petra Benedicta (Voir article dans [le Progrès](https://www.leprogres.fr/rhone-69-edition-ouest-lyonnais/2019/07/10/l-histoire-du-jour-la-pierre-qui-a-donne-son-nom-a-pierre-benite-existe-toujours))
1. Pierre-Bénite a 5 crèches, ce qui la place 26ème commune en nombre de crèches dans la métropole (Source : [data.grandlyon.com](https://download.data.grandlyon.com/ws/grandlyon/car_care.carcreche_latest/all.csv?maxfeatures=-1))
1. Pierre Bénite a 1 salle de cinéma (Source : [data.grandlyon.com](https://download.data.grandlyon.com/ws/grandlyon/adr_voie_lieu.adrcinema/all.csv?maxfeatures=-1))


In [31]:
creches_url = "https://download.data.grandlyon.com/ws/grandlyon/car_care.carcreche_latest/all.csv?maxfeatures=-1"
creches_file = "creches.csv"


In [32]:
creches = get_df(creches_url, creches_file)

Fichier déjà présent
DataFrame prête


In [33]:
creches

,nom,categorie,address,adresse,commune,insee,capacite,anneecreation,ctm,gestionnaire,groupe,datemaj,gid,lon,lat
0,Les Petits Futés,Association,"{'postalCode': '69250', 'streetAddress': '57 a...","57 avenue Gabriel Péri, 69250 Albigny-sur-Saône",Albigny-sur-Saône,69003,28,1992.0,Val de Saône,AGDS,NaN,2022-03-07,1,"4,8321708","45,8666595"
1,Arc En Ciel,Collectivité Locale,"{'postalCode': '69500', 'streetAddress': '63 r...","63 rue Christian Lacouture, 69500 Bron",Bron,69029,16,2010.0,Porte des Alpes,CCAS,NaN,2022-03-07,2,"4,9109369","45,7441375"
2,Clair De Lune,Hopital,"{'postalCode': '69500', 'streetAddress': 'CH d...","CH du Vinatier 95 bd Pinel, 69500 Bron",Bron,69029,70,1972.0,Porte des Alpes,CENTRE HOSPITALIER LE VINATIER,NaN,2022-03-07,3,"4,8941366","45,7420515"
3,Cocon Des Sens,Entreprise,"{'postalCode': '69500', 'streetAddress': '14 r...","14 rue Martin, 69500 Bron",Bron,69029,10,2019.0,Porte des Alpes,LA MAISON BLEUE MC EST 6,LA MAISON BLEUE SAS,2022-03-07,4,"4,8884619","45,7293739"
4,Croc' Noisette,Entreprise,"{'postalCode': '69500', 'streetAddress': 7 rue...","7 rue du 35ème Régiment d'Aviation, 69500 Bron",Bron,69029,20,2019.0,Porte des Alpes,LÉA ET LÉO SUD EST SARL,LÉA ET LÉO SAS,2022-03-07,5,"4,9286706","45,7381478"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
672,Tissot,Collectivité Locale,"{'postalCode': '69009', 'streetAddress': '1 ru...","1 rue Tissot, 69009 Lyon 9e Arrondissement",Lyon 9e Arrondissement,69389,56,1974.0,Lyon,VILLE DE LYON,NaN,2022-03-07,675,"4,8029704","45,7769441"
673,Un Air De Famille,Entreprise,"{'postalCode': '69009', 'streetAddress': '4 pl...","4 place Maurice Bariod, 69009 Lyon 9e Arrondis...",Lyon 9e Arrondissement,69389,10,2019.0,Lyon,UN AIR DE FAMILLE SARL,NaN,2022-03-07,676,"4,8249243","45,8028227"
674,Vanille Et Chocolat,Association,"{'postalCode': '69009', 'streetAddress': '26 a...","26 avenue Rosa Parks, 69009 Lyon 9e Arrondisse...",Lyon 9e Arrondissement,69389,20,1980.0,Lyon,CENTRE SOCIAL SAUVEGARDE,NaN,2022-03-07,677,"4,7916342","45,7868332"
675,Les Frippons,Association,"{'postalCode': '69330', 'streetAddress': 'plac...","place Général de Gaulle, 69330 Jonage",Jonage,69279,20,1972.0,Rhône Amont,CENTRE SOCIAUX DE GIVORS,NaN,2022-11-10,678,"5,0473148","45,7962016"


In [34]:
creches_classement = creches.groupby('commune').count()[['nom']].sort_values('nom', ascending=False)

In [35]:
creches_classement['position'] = range(1, 68)

In [36]:
creches_classement.loc[creches_classement.index.str.contains('Pierre')]

,nom,position
commune,,
Pierre-Bénite,5,26


In [37]:
cinema_url = "https://download.data.grandlyon.com/ws/grandlyon/adr_voie_lieu.adrcinema/all.csv?maxfeatures=-1"
cinema_filename = "cinemas.csv"

cinemas = get_df(cinema_url, cinema_filename)

Fichier déjà présent
DataFrame prête


In [38]:
cinemas[cinemas.commune.str.contains('Pierre')]

,nom,commune,insee,address,openinghours,capacite,nbecrans,cinema3d,telephone,url,source,gid,lon,lat
4,La Maison du Peuple,Pierre-Bénite,69152,"{'postalCode': '69310', 'streetAddress': '6 Pl...",NaN,88.0,1.0,False,NaN,NaN,Open Street Map,8,"4,8231402","45,7039147"


## Saint-Cyr au mont d'Or
### Les Questions
1. En 2020, les habitants de Saint-Cyr-au-Mont-d'Or ont consommé 2,5 fois moins d’électricité que les habitants de Lyon 9ème (Source : **data.grandlyon.com** > [énergie par parcelle](https://download.data.grandlyon.com/ws/rdata/nrj_energie.nrjcad_parcelles_2020/all.csv?maxfeatures=-1) et  [population](https://download.data.grandlyon.com/ws/grandlyon/ter_territoire.terpopulation_latest/all.csv?maxfeatures=-1))
1. Les habitants de Saint-Cyr-au-Mont-d'Or sont appelés les Saint-Cyrôts, tandis que les habitants des différentes communes appelées Saint-Cyr sont appelés les Saint-Cyriens (Source : [habitants.fr](https://www.habitants.fr/saint-cyr-au-mont-d-or/services-publics)]
1. Saint-Cyr-au-Mont-d'Or n'a pas de salle de cinéma (Source : [data.grandlyon.com](https://download.data.grandlyon.com/ws/grandlyon/adr_voie_lieu.adrcinema/all.csv?maxfeatures=-1))

In [39]:
energie_parcelle_url = "https://download.data.grandlyon.com/ws/rdata/nrj_energie.nrjcad_parcelles_2020/all.csv?maxfeatures=-1"
energie_parcelle_filename = "energie_parcelle.csv"

In [40]:
population_url = "https://download.data.grandlyon.com/ws/grandlyon/ter_territoire.terpopulation_latest/all.csv?maxfeatures=-1"
population_filename = "population.csv"

In [41]:
population = get_df(population_url, population_filename)

Fichier déjà présent
DataFrame prête


In [42]:
energie = get_df(energie_parcelle_url, energie_parcelle_filename)

Fichier déjà présent
DataFrame prête


In [43]:
energie.conso_elec = energie.conso_elec.str.replace(',', '.').apply(lambda s: float(s))
energie.conso_gaz = energie.conso_gaz.str.replace(',', '.').apply(lambda s: float(s))

In [44]:
population_2018 = population[['commune', 'pop2018']]

In [45]:
energie_classement = energie.groupby('commune').sum(numeric_only=True)[['conso_elec', 'conso_gaz']]

In [46]:
energie_classement = energie_classement.reset_index()

In [47]:
energie_classement = energie_classement.merge(population_2018)

In [48]:
energie_classement["conso_elec_par_population"] = energie_classement.conso_elec / energie_classement.pop2018

In [49]:
energie_classement = energie_classement.sort_values(by="conso_elec_par_population").reset_index(drop='index')

In [50]:
energie_classement

,commune,conso_elec,conso_gaz,pop2018,conso_elec_par_population
0,Poleymieux-au-Mont-d'Or,58.13,0.00,1383,0.042032
1,Cailloux-sur-Fontaines,228.40,852.17,2875,0.079443
2,Curis-au-Mont-d'Or,145.07,165.96,1202,0.120691
3,Charly,649.97,963.24,4691,0.138557
4,Sathonay-Village,481.52,510.84,2422,0.198811
...,...,...,...,...,...
62,Lyon 2e Arrondissement,137695.89,76027.35,31123,4.424249
63,Corbas,51963.41,32632.21,11180,4.647890
64,Limonest,17637.84,2616.22,3750,4.703424
65,Feyzin,104102.20,18687.24,9931,10.482550


In [51]:
energie_classement[energie_classement.commune.str.contains('yr')]

,commune,conso_elec,conso_gaz,pop2018,conso_elec_par_population
26,Saint-Cyr-au-Mont-d'Or,5908.63,5843.22,5862,1.007955


In [52]:
cinemas[cinemas.commune.str.contains('yr')]

,nom,commune,insee,address,openinghours,capacite,nbecrans,cinema3d,telephone,url,source,gid,lon,lat


## Neuville sur Saone
### Les Questions
1. À la Révolution, elle s'appelle temporairement Marat-sur-Saône avant de prendre le nom de Neuville-sur-Saône. (Source : [École des Hautes Études en Sciences Sociales](http://cassini.ehess.fr/fr/html/fiche.php?select_resultat=24928))
1. Neuville sur Saône dispose de 29 établissements recevant du public, ce qui la place 3ème ex-aequo avec Corbas parmi les 6 communes présentes. (Source : [data.grandlyon.com](https://download.data.grandlyon.com/ws/rdata/sdmis.erp/all.csv?maxfeatures=-1))
1. Il y a 2 Quartiers prioritaires de la politique de la ville (QPV) à Neuville-sur-Saône. (Source : [data.grandlyon.com](https://download.data.grandlyon.com/ws/grandlyon/ter_territoire.terqpvqva_latest/all.csv?maxfeatures=-1))

In [53]:
erp_url = "https://download.data.grandlyon.com/ws/rdata/sdmis.erp/all.csv?maxfeatures=-1"
erp_filename = "erp.csv"

In [54]:
erp = get_df(erp_url, erp_filename)

Fichier déjà présent
DataFrame prête


In [55]:
erp[erp.commune.str.contains('NEUVILLE')]

,code,libelle,nature,categorie,type,types_secondaires,hebergement,effectif_total_reglementaire,adresse,code_postal,insee,commune,gid,lon,lat
1470,E14300001000,"Stade Municipal ""Jean Oboussier""",ERP,2.0,PA,NaN,NaN,1200,route de Lyon,69250,69143,NEUVILLE SUR SAÔNE,1427,"4,83818069","45,869217"
1471,E14300002000,Maison de la Musique,ERP,4.0,R,NaN,NaN,300,37 rue Victor Hugo,69250,69143,NEUVILLE SUR SAÔNE,1428,"4,84202063","45,87692191"
1472,E14300003000,Magasin Gifi,ERP,2.0,M,NaN,NaN,850,22 avenue Carnot,69250,69143,NEUVILLE SUR SAÔNE,1429,"4,83507521","45,88313625"
1473,E14300004000,Supermarché Casino,ERP,3.0,M,NaN,NaN,647,4-6 rue Pollet,69250,69143,NEUVILLE SUR SAÔNE,1430,"4,84336919","45,87597994"
1474,E14300007000,"Hôtel-restaurant ""O Petit Nice""",ERP,5.0,O,NaN,oui,50,10 quai Armand Barbès,69250,69143,NEUVILLE SUR SAÔNE,1431,"4,83409493","45,88018137"
1475,E14300009000,Maison d'enfants Balmont,ERP,4.0,RH,NaN,oui,60,46 avenue Auguste Wissel,69250,69143,NEUVILLE SUR SAÔNE,1432,"4,849173","45,88361332"
1476,E14300010000,GS Primaire de la Tatière,ERP,4.0,R,NaN,NaN,111,rue Pierre Dupont,69250,69143,NEUVILLE SUR SAÔNE,1433,"4,84108902","45,87346634"
1477,E14300011000,"Espace associatif ""Margerand"" - Bâtiment A",ERP,3.0,L,NaN,NaN,323,11 avenue Gambetta,69250,69143,NEUVILLE SUR SAÔNE,1434,"4,84123475","45,87242614"
1478,E14300013000,Espace Jean Vilar,ERP,2.0,L,NaN,NaN,1065,place Charles De Gaulle,69250,69143,NEUVILLE SUR SAÔNE,1435,"4,8378229","45,87875852"
1479,E14300014000,Maison des Jeunes et de la Culture,ERP,3.0,L,NaN,NaN,608,15 place du 8 Mai 1945,69250,69143,NEUVILLE SUR SAÔNE,1436,"4,84077686","45,87794318"


In [56]:
erp_classement = erp.groupby('commune').count().sort_values('code', ascending=False)[['code']].reset_index()

In [57]:
erp_classement['position'] = erp_classement.index + 1

In [58]:
COMMUNES = ['BRON', 'CORBAS', 'LA TOUR DE SALVAGNY', "PIERRE BÉNITE", "SAINT CYR AU MONT D'OR", "NEUVILLE SUR SAÔNE", "FRANCHEVILLE"]

In [59]:
erp_classement.loc[erp_classement.commune.isin(COMMUNES), :]

,commune,code,position
7,BRON,154,8
27,PIERRE BÉNITE,43,28
34,FRANCHEVILLE,33,35
36,NEUVILLE SUR SAÔNE,29,37
37,CORBAS,29,38
52,SAINT CYR AU MONT D'OR,22,53
58,LA TOUR DE SALVAGNY,19,59


In [60]:
quartiers_prioritaires_url = "https://download.data.grandlyon.com/ws/grandlyon/ter_territoire.terqpvqva_latest/all.csv?maxfeatures=-1"
quartiers_prioritaires_filename = "quartiers_prioritaires.csv"

In [61]:
quartiers = get_df(quartiers_prioritaires_url, quartiers_prioritaires_filename)

Fichier déjà présent
DataFrame prête


In [62]:
quartiers.head(6)

,gid,nom,code,type,communes
0,1,Les Buers Nord,QP069024,QPV,"[{'insee': '69266', 'commune': 'Villeurbanne'}]"
1,2,Mermoz,QP069036,QPV,"[{'insee': '69388', 'commune': 'Lyon 8e Arrond..."
2,3,Sud,QP069019,QPV,"[{'insee': '69256', 'commune': 'Vaulx-en-Velin'}]"
3,4,Ville Nouvelle,QP069028,QPV,"[{'insee': '69286', 'commune': 'Rillieux-la-Pa..."
4,5,Bellevue,QP069029,QPV,"[{'insee': '69290', 'commune': 'Saint-Priest'}]"
5,6,Etats-Unis - Langlet Santy,QP069035,QPV,"[{'insee': '69388', 'commune': 'Lyon 8e Arrond..."


In [63]:
def extract_communes(s):
    dict_list = eval(s)
    communes_list = [d.get('commune') for d in dict_list]
    return " ".join(communes_list)

In [64]:
quartiers.communes = quartiers.communes.apply(extract_communes)

In [65]:
quartiers[quartiers.communes.str.startswith('Pier')]

,gid,nom,code,type,communes
16,17,Hautes Roches,QP069014,QPV,Pierre-Bénite
98,1062,Haute Roche,QP069014,QVA,Pierre-Bénite


## Francheville

1. Il y a 1 collège public et 1 musée à Francheville. Source : [data.grandlyon.com](https://download.data.grandlyon.com/ws/rdata/edu_education.college/all.csv?maxfeatures=-1)
1. Francheville a 11 crèches a Francheville, ce qui la classe seconde parmi les communes réunies après Bron. Source : [data.grandlyon.com](https://download.data.grandlyon.com/ws/grandlyon/car_care.carcreche_latest/all.csv?maxfeatures=-1)
1. La commune est partagée en trois quartiers : le Bourg, le Châter et Bel Air)

In [66]:
creches_url

'https://download.data.grandlyon.com/ws/grandlyon/car_care.carcreche_latest/all.csv?maxfeatures=-1'

In [67]:
communes = ['Bron',
            'Corbas',
            'La Tour-de-Salvagny',
            'Pierre-Bénite',
            "Saint-Cyr-au-Mont-d'Or",
            'Neuville-sur-Saône',
            'Francheville'
           ]

In [68]:
colleges_url = "https://download.data.grandlyon.com/ws/rdata/edu_education.college/all.csv?maxfeatures=-1"
colleges_filename = "colleges.csv"

In [69]:
colleges = get_df(colleges_url, colleges_filename)

Fichier déjà présent
DataFrame prête


In [70]:
nb_colleges = nb_par_commune(colleges)

In [71]:
nb_colleges

,nombre
commune,
Villeurbanne,17
Lyon 5e Arrondissement,8
Lyon 7e Arrondissement,8
Vénissieux,7
Lyon 3e Arrondissement,7
...,...
Saint-Cyr-au-Mont-d'Or,1
Saint-Didier-au-Mont-d'Or,1
Saint-Fons,1


In [72]:
nb_colleges[nb_colleges.index.isin(communes)]

,nombre
commune,
Bron,3
Pierre-Bénite,2
Neuville-sur-Saône,2
Francheville,1
Corbas,1
Saint-Cyr-au-Mont-d'Or,1


In [73]:
creches[creches.commune=="Francheville"]

,nom,categorie,address,adresse,commune,insee,capacite,anneecreation,ctm,gestionnaire,groupe,datemaj,gid,lon,lat
81,Air D'enfance,Collectivité Locale,"{'postalCode': '69340', 'streetAddress': '4 ru...","4 rue de la Chapelle de Bel Air, 69340 Franche...",Francheville,69089,30,1982.0,Val d'Yzeron,COMMUNE DE FRANCHEVILLE,NaN,2022-03-07,82,"4,7566557","45,7465405"
82,Crèche Du Pont D'alaï,Association,"{'postalCode': '69340', 'streetAddress': '35 B...","35 B chemin de la Poterie, 69340 Francheville",Francheville,69089,30,1982.0,Val d'Yzeron,ALFA3A,NaN,2022-03-07,83,"4,7613447","45,750756"
83,Franch' Mômes,Collectivité Locale,"{'postalCode': '69340', 'streetAddress': '93 g...","93 grande Rue, 69340 Francheville",Francheville,69089,15,1992.0,Val d'Yzeron,COMMUNE DE FRANCHEVILLE,NaN,2022-03-07,84,"4,7627712","45,7369938"
84,Franch'bambins,Collectivité Locale,"{'postalCode': '69340', 'streetAddress': 1 rue...","1 rue de l'Aubier, 69340 Francheville",Francheville,69089,40,1994.0,Val d'Yzeron,COMMUNE DE FRANCHEVILLE,NaN,2022-03-07,85,"4,7724805","45,7372892"
85,Graines D'écolos,Entreprise,"{'postalCode': '69340', 'streetAddress': '1 bi...","1 bis chemin de Torey, 69340 Francheville",Francheville,69089,10,2015.0,Val d'Yzeron,GRAINES D'ÉCOLOS SARL,NaN,2022-03-07,86,"4,7560023","45,7489643"
86,Graines D'écolos 4,Entreprise,"{'postalCode': '69340', 'streetAddress': 5 all...","5 allée de l'Hôtel de Ville, 69340 Francheville",Francheville,69089,10,2020.0,Val d'Yzeron,GRAINES D'ÉCOLOS SARL,NaN,2022-03-07,87,"4,7647981","45,7360805"
87,La Cabane,Entreprise,"{'postalCode': '69340', 'streetAddress': 4 all...","4 allée de l'Expansion, 69340 Francheville",Francheville,69089,16,2019.0,Val d'Yzeron,PEOPLE AND BABY SAS,PEOPLE AND BABY SAS,2022-03-07,88,"4,7544409","45,74993"
88,Le Jardin D'enfants Du Pont D'alaï,Association,"{'postalCode': '69340', 'streetAddress': '35 A...","35 A chemin de la Poterie, 69340 Francheville",Francheville,69089,24,2013.0,Val d'Yzeron,ALFA3A,NaN,2022-03-07,89,"4,7613447","45,750756"
89,Les Marsupiaux Bel Air,Entreprise,"{'postalCode': '69340', 'streetAddress': '8 im...","8 impasse Maillabert, 69340 Francheville",Francheville,69089,10,2017.0,Val d'Yzeron,LES MARSUPIAUX SARL,NaN,2022-03-07,90,"4,7508552","45,7445006"
90,Little Tribu Les Petits Sioux,Entreprise,"{'postalCode': '69340', 'streetAddress': '82 a...","82 avenue du Chater, 69340 Francheville",Francheville,69089,11,2016.0,Val d'Yzeron,LITTLE TRIBU SAS,NaN,2022-03-07,91,"4,7742248","45,7371376"


In [74]:
musees_url = "https://download.data.grandlyon.com/ws/grandlyon/adr_voie_lieu.adrmusee/all.csv?maxfeatures=-1"
musees_filename = "musees.csv"

In [75]:
musees = get_df(musees_url, musees_filename)

Fichier déjà présent
DataFrame prête


In [76]:
nb_musees = nb_par_commune(musees)

In [77]:
nb_musees[nb_musees.index.isin(communes)]

,nombre
commune,
Corbas,1
Francheville,1


In [78]:
musees[musees.commune == "Francheville"]

,nom,commune,insee,address,openinghours,acceshandi,telephone,url,wikipedia,source,gid,lon,lat
2,Centre d'art contemporain du Fort du Bruissin,Francheville,69089,"{'postalCode': '69340', 'streetAddress': '7 Ru...",[],NaN,NaN,NaN,NaN,Open Street Map,18,"4,7385345","45,7305319"


In [79]:
nb_musees

,nombre
commune,
Lyon 5e Arrondissement,5
Lyon 2e Arrondissement,4
Lyon 4e Arrondissement,4
Lyon 7e Arrondissement,4
Lyon 8e Arrondissement,4
Lyon 3e Arrondissement,2
Lyon 9e Arrondissement,2
Charbonnières-les-Bains,1
Corbas,1


In [80]:
nb_creches = nb_par_commune(creches)

In [81]:
nb_creches[nb_creches.index.isin(communes)]

,nombre
commune,
Bron,18
Francheville,11
Pierre-Bénite,5
Saint-Cyr-au-Mont-d'Or,3
Neuville-sur-Saône,3
Corbas,3
La Tour-de-Salvagny,1
